# FAISS VectorStore
1. CSV에서 한 행씩 chunking

1. CSVLoader() 로 csv 파일 로드
2. xml document 생성
3. TextSplitter로 분할 : chunk_size=600, 
4. chroma db 생성(우선 10개만)
   - splited text로 db 생성
   - embedding : OpenAIEmbedding()
5. query 테스트

In [1]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [2]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH23-MyProject")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH23-MyProject


In [ ]:
# CSV(공고데이터) 로드
from langchain_community.document_loaders.csv_loader import CSVLoader

# CSV 파일 경로
loader = CSVLoader(
    file_path="./data/announcement.csv",
    csv_args={
        "delimiter": ",",  # 구분자
        "quotechar": '"',  # 인용 부호 문자
        "fieldnames": [
            "Sequence",  # 순번
            "Registration number",  # 공고번호
            "Announcement name",  # 공고명
            "Support areas",  # 지원분야
            "Region",  # 지역
            "Target",  # 지원대상
            "Target age",  # 대상연령
            "Application period",  # 접수기간
            "Entrepreneurial history",  # 업력
            "Institution name",  # 기관명
            "Organization classification",  # 기관구분 : 공공, 민간, 교육
            "Department in charge",  # 담당부서
            "Announcement number",  # 공고 제 호
            "Announcement contents",  # 공고내용
            "Registration date",  # 공고등록일
            "Announcement registrar name",  # 공고 기업명
            "How to apply work-in",  # 신청방법 : 방문
            "How to apply By mail",  # 신청방법 : 우편
            "How to apply By Fax",  # 신청방법 : FAX
            "How to apply By email",  # 신청방법 : email
            "How to apply online",  # 신청방법 : 온라인
            "How to apply other",  # 신청방법 : 기타
            "Who to apply for",  # 신청대상
            "Excluded from application",  # 제외대상
            "Summary",  # 공고명 + 공고내용
        ],  # 필드 이름
    },
)

# 데이터 로드
docs = loader.load()

# 데이터 출력
print(docs[1].page_content)

In [5]:
len(docs[2].page_content)

1129

In [ ]:
# XML 변환
xml_docs = []
for doc in docs[1:]:
    row = doc.page_content.split("\n")
    row_str = "<row>"
    for element in row:
        splitted_element = element.split(":")
        value = splitted_element[-1]
        col = ":".join(splitted_element[:-1])
        row_str += f"<{col}>{value.strip()}</{col}>"
    row_str += "</row>"
    xml_docs.append(row_str)

In [ ]:
# 리스트를 텍스트 파일에 저장
with open("./xml_doc.txt", "w") as f:
    for item in xml_docs:
        f.write(f"{item}\n")

In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma

In [ ]:
# 텍스트 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)

loader1 = TextLoader("./xml_doc.txt")

# 문서 분할
split_doc = loader1.load_and_split(text_splitter)

# 문서 개수 확인
len(split_doc)

In [ ]:
split_doc[:10]

In [ ]:
from langchain.retrievers import EnsembleRetriever
from langchain.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

In [ ]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
faiss_vectorstore = FAISS.from_documents(documents=split_doc, embedding=embeddings)

In [ ]:
# FAISS index 저장
faiss_vectorstore.save_local("./faiss500_index")

In [ ]:
# FAISS index 불러오기
faiss_index_path = "./faiss500_index"
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

faiss_db = FAISS.load_local(
    faiss_index_path, embeddings, allow_dangerous_deserialization=True
)

In [ ]:
query1 = "나는 서울에 살고있어.. 창업을 할려고 하는데 어떤 지원을 받을 수 있을까"
query2 = "예비창업패키지에 대해 설명해줘"
# result = faiss_vectorstore.similarity_search(query1, k=3)
result = faiss_db.similarity_search(query2, k=3)
print(result[0].page_content)
print(result[1].page_content)

![](./images/chroma-06.png)